In [2]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

# Start timer.
t0 = time.time() # TIME.TIME GRABS CURRENT TIME OF RUNNINGN THIS CODE

# Import word vectors into "model."
model = gensim.models.KeyedVectors.load_word2vec_format('./lexvec.enwiki+newscrawl.300d.W.pos.vectors')

# Print results of timer.
print(time.time() - t0)

C:\Users\Michelle\Anaconda3\envs\dsi\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


106.21171402931213


## Read `train` pickle: includes tweets from Hurricane Harvey, Hurricane Maria, and Hurricane Irma
 - This dataset includes data labeled for images

In [3]:
train_1 = pd.read_pickle('./train.pkl')

In [4]:
train_1.head(2)

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path,event
0,905274232590004225,901671684478029825_0,not_informative,1.0000,informative,1.0,not_relevant_or_cant_judge,1.0000,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",http://pbs.twimg.com/media/DINhGs8XcAE1H7q.jpg,data_image/hurricane_harvey/27_8_2017/90167168...,harvey
1,901646074527535105,901646074527535105_0,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6728,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxh_xWAAAfJDY.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey


In [5]:
train_1.shape

(13530, 16)

### Remove duplicates

In [6]:
train_1.drop_duplicates(inplace=True)

In [7]:
train_1.shape

(13530, 16)

### Binarize target column `text_info`

In [8]:
train_1['text_info'].value_counts()

informative                9742
not_informative            3784
dont_know_or_cant_judge       4
Name: text_info, dtype: int64

In [9]:
train_1 = train_1[train_1.text_info != 'dont_know_or_cant_judge']

In [10]:
train_1['text_info'].value_counts()

informative        9742
not_informative    3784
Name: text_info, dtype: int64

In [11]:
train_1['informative'] = train_1.text_info.map(lambda x: 1 if x == 'informative' else 0)
train_1.head()

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path,event,informative
0,905274232590004225,901671684478029825_0,not_informative,1.0000,informative,1.0,not_relevant_or_cant_judge,1.0000,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",http://pbs.twimg.com/media/DINhGs8XcAE1H7q.jpg,data_image/hurricane_harvey/27_8_2017/90167168...,harvey,0
1,901646074527535105,901646074527535105_0,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6728,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxh_xWAAAfJDY.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey,1
2,901646074527535105,901646074527535105_1,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxiisXYAAokz_.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey,1
3,901646074527535105,901646074527535105_2,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6528,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxjSUWAAEIaWH.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey,1
4,901646074527535105,901646074527535105_3,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6812,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxkWJXgAANF-E.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey,1


In [12]:
train_1['tweet_text'].iloc[0]

"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure the victims of Harvey will appreciate it greatly https://t.co/daPhXZvhuY"

### Extract columns for X

In [13]:
train_1_for_X = train_1[['tweet_text', 'informative']]

In [14]:
train_1_for_X.head()

,tweet_text,informative
0,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",0
1,RT @ajwamood: #ajwamood : Harvey the first maj...,1
2,RT @ajwamood: #ajwamood : Harvey the first maj...,1
3,RT @ajwamood: #ajwamood : Harvey the first maj...,1
4,RT @ajwamood: #ajwamood : Harvey the first maj...,1


In [15]:
train_1_for_X.rename(columns={'tweet_text': 'train_text'}, inplace=True)
train_1_for_X.head()

C:\Users\Michelle\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,train_text,informative
0,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",0
1,RT @ajwamood: #ajwamood : Harvey the first maj...,1
2,RT @ajwamood: #ajwamood : Harvey the first maj...,1
3,RT @ajwamood: #ajwamood : Harvey the first maj...,1
4,RT @ajwamood: #ajwamood : Harvey the first maj...,1


## Read in `train_2` pickle: includes tweets from Hurricane Sandy and Hurrican Joplin
 - This dataset includes labeled data for text

In [16]:
train_2 = pd.read_pickle('./train2.pkl')

In [17]:
train_2.head(2)

,_golden,_last_judgment_at,_trusted_judgments,_unit_state,choose_one,choose_one:confidence,choose_one_gold,event,nil,predicted,text_no_rt,tweet,user,informative
_unit_id,,,,,,,,,,,,,,
204690712,False,10/6/2012 03:35:03,4,finalized,Informative (Direct or Indirect),0.5133,NaN,joplin,NaN,Informative (Indirect),Cruuuuud. Tornado warning for Denton County. A...,Cruuuuud. Tornado warning for Denton County. A...,NaN,1
204690713,True,10/6/2012 03:25:33,129,golden,Informative (Indirect),0.4781,Informative (Indirect),joplin,NaN,Informative (Indirect),Tornado WARNING for Tarrant county. Storm spot...,Tornado WARNING for Tarrant county. Storm spot...,NaN,1


In [18]:
train_2.shape

(3384, 14)

### Remove duplicates

In [19]:
train_2.drop_duplicates(inplace=True)

In [20]:
train_2.shape

(3384, 14)

### Extract columns for X

In [21]:
train_2_for_X = train_2[['text_no_rt', 'informative']]

In [22]:
train_2_for_X.head()

,text_no_rt,informative
_unit_id,,
204690712,Cruuuuud. Tornado warning for Denton County. A...,1
204690713,Tornado WARNING for Tarrant county. Storm spot...,1
204690714,Tornado on the ground in Azle headed right tow...,1
204690715,US tornado toll rises to 122 (AFP) http://dlvr...,1
204690716,US tornado toll rises to 122 (AFP) http://dlvr...,1


In [23]:
train_2_for_X.rename(columns = {'text_no_rt': 'train_text'}, inplace=True)
train_2_for_X.reset_index(drop=True, inplace=True)
train_2_for_X.head()

C:\Users\Michelle\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,train_text,informative
0,Cruuuuud. Tornado warning for Denton County. A...,1
1,Tornado WARNING for Tarrant county. Storm spot...,1
2,Tornado on the ground in Azle headed right tow...,1
3,US tornado toll rises to 122 (AFP) http://dlvr...,1
4,US tornado toll rises to 122 (AFP) http://dlvr...,1


## Merge Dataframes

In [24]:
train = pd.concat([train_1_for_X, train_2_for_X])

In [25]:
train.shape

(16910, 2)

In [26]:
train.head()

,train_text,informative
0,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",0
1,RT @ajwamood: #ajwamood : Harvey the first maj...,1
2,RT @ajwamood: #ajwamood : Harvey the first maj...,1
3,RT @ajwamood: #ajwamood : Harvey the first maj...,1
4,RT @ajwamood: #ajwamood : Harvey the first maj...,1


In [27]:
train.isnull().sum()

train_text     0
informative    0
dtype: int64

In [28]:
train.dtypes

train_text     object
informative     int64
dtype: object

In [29]:
train.to_pickle('combined_train.pkl')

In [30]:
train['train_text'].iloc[1]

'RT @ajwamood: #ajwamood : Harvey the first major hurricane to strike the #US since 2005 , causes at least 1 fatality https://t.co/RvDkebyAAC'

## Clean data

### Remove Punctutation

In [31]:
train['train_text'] = train['train_text'].str.replace("[^a-zA-Z]", " ")

### Remove HTML Code artifacts

### Define X

In [63]:
X = train_1['tweet_text']

In [64]:
X.head()

0    CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...
1    RT @ajwamood: #ajwamood : Harvey the first maj...
2    RT @ajwamood: #ajwamood : Harvey the first maj...
3    RT @ajwamood: #ajwamood : Harvey the first maj...
4    RT @ajwamood: #ajwamood : Harvey the first maj...
Name: tweet_text, dtype: object

In [65]:
X.iloc[10]

'RT @MSNBC: Side by side satellite images compare the size of Hurricane Harvey to Hurricane Katrina in 2005. https://t.co/8LemA9Ju1v'

### TEST FOR INPUT INTO WORD2VEC

In [71]:
from nltk.tokenize import TweetTokenizer

In [72]:
tweets_cleaned = pd.read_pickle('./tweets_cleaned.pkl')

In [73]:
tweets_cleaned.head()

,tweet_text,informative
0,congrats on hitting yoir goal guys i m sure t...,0
1,#ajwamood harvey the first major hurricane...,1
5,when we get back to schs after harvey hits,0
9,not always good when your city shows up on a s...,1
10,side by side satellite images compare the si...,1


In [75]:
tknzr = TweetTokenizer()

In [76]:
tokens = []
for tweet in tweets_cleaned.tweet_text:
    tokens.append(tknzr.tokenize(tweet))

### Word2Vec Model

In [77]:
model = Word2Vec(tokens,      # Corpus of data.
                 size=100,    # How many dimensions do you want in your word vector?
                 window=5,    # How many "context words" do you want?
                 min_count=1, # Ignores words below this threshold.
                 sg=0,        # SG = 1 uses SkipGram, SG = 0 uses CBOW (default).
                 workers=4)   # Number of "worker threads" to use (parallelizes process).

C:\Users\Michelle\Anaconda3\envs\dsi\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [78]:
model.most_similar('food', topn=25)

C:\Users\Michelle\Anaconda3\envs\dsi\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('their', 0.9999050498008728),
 ('volunteers', 0.9998874664306641),
 ('items', 0.9998669624328613),
 ('up', 0.9998631477355957),
 ('community', 0.9998438954353333),
 ('thanks', 0.9998221397399902),
 ('fema', 0.9998189210891724),
 ('#hurricaneharvey', 0.9998002052307129),
 ('federal', 0.9997961521148682),
 ('continue', 0.999793291091919),
 ('family', 0.9997842311859131),
 ('tax', 0.9997715353965759),
 ('residents', 0.9997698664665222),
 ('emergency', 0.9997696280479431),
 ('flood', 0.9997669458389282),
 ('crews', 0.9997556805610657),
 ('kids', 0.9997546076774597),
 ('free', 0.999754011631012),
 ('work', 0.9997531771659851),
 ('raising', 0.999752402305603),
 ('debt', 0.999747633934021),
 ('visit', 0.9997398257255554),
 ('join', 0.9997353553771973),
 ('k', 0.9997285008430481),
 ('contract', 0.9997202157974243)]

In [23]:
model.most_similar('food', topn=25)

[('foods', 0.6212202310562134),
 ('seafood', 0.6099287867546082),
 ('meals', 0.6038150191307068),
 ('foodstuffs', 0.5895399451255798),
 ('meat', 0.5782395601272583),
 ('nutritious', 0.5607747435569763),
 ('meal', 0.5420812368392944),
 ('groceries', 0.5333590507507324),
 ('snack', 0.5314205288887024),
 ('nutrition', 0.5273828506469727),
 ('grocery', 0.5240978002548218),
 ('poultry', 0.5150964260101318),
 ('snacks', 0.5125464200973511),
 ('necessities', 0.5027074813842773),
 ('beverage', 0.49984222650527954),
 ('foodstuff', 0.49397554993629456),
 ('takeaway', 0.4935123026371002),
 ('dairy', 0.48863810300827026),
 ('vegetables', 0.48462727665901184),
 ('nutritional', 0.4812769293785095),
 ('ingredients', 0.47778424620628357),
 ('perishable', 0.4772763252258301),
 ('bottled', 0.4752594232559204),
 ('calorie', 0.4723014831542969),
 ('sustenance', 0.4694999158382416)]

In [24]:
model.most_similar('water', topn=25)

[('seawater', 0.6570144891738892),
 ('groundwater', 0.6502763032913208),
 ('sewage', 0.6035518646240234),
 ('potable', 0.587207555770874),
 ('wastewater', 0.5817205905914307),
 ('rainwater', 0.5797319412231445),
 ('stormwater', 0.5383915305137634),
 ('sewer', 0.5179448127746582),
 ('effluent', 0.5174242258071899),
 ('irrigation', 0.5160751938819885),
 ('sediment', 0.5143337249755859),
 ('floodwater', 0.5104843378067017),
 ('sewerage', 0.5085800886154175),
 ('aquifers', 0.508300244808197),
 ('brackish', 0.5063480138778687),
 ('liquid', 0.4911225140094757),
 ('bottled', 0.4858362674713135),
 ('evaporation', 0.4840138852596283),
 ('land', 0.47957533597946167),
 ('saltwater', 0.47671690583229065),
 ('seepage', 0.47353124618530273),
 ('moisture', 0.4711664617061615),
 ('oxygen', 0.46662917733192444),
 ('contaminate', 0.4646880030632019),
 ('polluted', 0.46368181705474854)]

In [26]:
model.most_similar('medical', topn=25)

[('veterinary', 0.6589906811714172),
 ('dental', 0.6145017147064209),
 ('psychiatric', 0.5920779705047607),
 ('health', 0.5920690298080444),
 ('outpatient', 0.5774596929550171),
 ('hospital', 0.5629482269287109),
 ('medicine', 0.5626959800720215),
 ('surgical', 0.5600948333740234),
 ('nursing', 0.5600548386573792),
 ('clinical', 0.5514633655548096),
 ('pediatric', 0.5513782501220703),
 ('physicians', 0.5486127138137817),
 ('biomedical', 0.5468931794166565),
 ('orthopedic', 0.5442311763763428),
 ('radiology', 0.5297563076019287),
 ('palliative', 0.5220820903778076),
 ('healthcare', 0.5152167081832886),
 ('physician', 0.5142868757247925),
 ('hospitals', 0.51274573802948),
 ('orthopaedic', 0.5115314722061157),
 ('geriatric', 0.5042352676391602),
 ('doctors', 0.5006060004234314),
 ('dentistry', 0.5000715851783752),
 ('anesthesiology', 0.49136239290237427),
 ('pharmacy', 0.4911144971847534)]

In [27]:
model.most_similar('shelter', topn=25)

[('shelters', 0.8452489972114563),
 ('refuge', 0.6221227645874023),
 ('sheltering', 0.5715663433074951),
 ('homeless', 0.5680542588233948),
 ('tents', 0.5575913190841675),
 ('sheltered', 0.5509488582611084),
 ('evacuees', 0.5110695362091064),
 ('accommodation', 0.5065698027610779),
 ('evacuate', 0.4875040650367737),
 ('blankets', 0.4782549738883972),
 ('huts', 0.4649191200733185),
 ('sanctuary', 0.4641597270965576),
 ('needy', 0.4640933871269226),
 ('campsite', 0.462602823972702),
 ('refuges', 0.46182799339294434),
 ('respite', 0.4608321487903595),
 ('assistance', 0.45172184705734253),
 ('destitute', 0.44732755422592163),
 ('sustenance', 0.4466785490512848),
 ('tarps', 0.44646328687667847),
 ('shacks', 0.44627612829208374),
 ('orphanage', 0.44555678963661194),
 ('makeshift', 0.4432254135608673),
 ('tarpaulins', 0.4413571059703827),
 ('homes', 0.43969133496284485)]

In [28]:
model.most_similar('electricity', topn=25)

[('energy', 0.6751303672790527),
 ('megawatts', 0.6466302871704102),
 ('power', 0.6386343836784363),
 ('renewable', 0.6368008852005005),
 ('renewables', 0.6112408638000488),
 ('generators', 0.5911064743995667),
 ('utilities', 0.5753559470176697),
 ('hydropower', 0.5750429630279541),
 ('kilowatts', 0.5726759433746338),
 ('hydroelectricity', 0.5711821913719177),
 ('gas', 0.5674902200698853),
 ('kilowatt', 0.5663821697235107),
 ('baseload', 0.5652947425842285),
 ('kwh', 0.5643448829650879),
 ('heating', 0.5573583245277405),
 ('fuel', 0.5544668436050415),
 ('mwh', 0.5429953336715698),
 ('gigawatts', 0.541540265083313),
 ('sewerage', 0.5400328636169434),
 ('biomass', 0.5393158793449402),
 ('supply', 0.5359727144241333),
 ('hydroelectric', 0.5322878360748291),
 ('megawatt', 0.5314869284629822),
 ('photovoltaic', 0.5290755033493042),
 ('biogas', 0.5242639780044556)]

In [29]:
model.most_similar('power', topn=25)

[('electricity', 0.6386343836784363),
 ('kilowatts', 0.5114115476608276),
 ('megawatts', 0.5108576416969299),
 ('generators', 0.5044735670089722),
 ('powers', 0.5036338567733765),
 ('energy', 0.49356985092163086),
 ('hydroelectric', 0.4765773415565491),
 ('clout', 0.469901978969574),
 ('hydropower', 0.46867555379867554),
 ('renewables', 0.44705966114997864),
 ('renewable', 0.44471052289009094),
 ('turbines', 0.43001532554626465),
 ('baseload', 0.42854470014572144),
 ('utilities', 0.4276842176914215),
 ('capacity', 0.4268409013748169),
 ('levers', 0.4164648652076721),
 ('substations', 0.4157220721244812),
 ('electric', 0.41195422410964966),
 ('control', 0.41183674335479736),
 ('kw', 0.4112171232700348),
 ('output', 0.4105415940284729),
 ('mw', 0.40955716371536255),
 ('hydroelectricity', 0.4082127809524536),
 ('generating', 0.40796446800231934),
 ('photovoltaic', 0.4064619839191437)]